In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Sequential, Input
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from generator import Generator

In [64]:
def plot():
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plot()
    
def pre_process_isotherm(isotherm):
    isotherm -= min(isotherm)
    isotherm /= max(isotherm)
    return isotherm
    
def load_dataset(path):
    min_exp_pressure_i = 40
    max_exp_pressure_i =  547 # silcia 458 # carbon 547
    with open(path, 'rb') as f:
            dataset = np.load(f)
            isotherm_data = dataset["isotherm_data"]
            pore_distribution_data = dataset["pore_distribution_data"]
    x = np.empty((isotherm_data.shape[0], (-min_exp_pressure_i + max_exp_pressure_i)))
    y = np.empty(pore_distribution_data.shape)
    for i in range(len(isotherm_data)):
        isotherm = pre_process_isotherm(isotherm_data[i][min_exp_pressure_i:max_exp_pressure_i])
        pore_distribution = pre_process_isotherm(pore_distribution_data[i])
        x[i] = isotherm
        y[i] = pore_distribution
    x, y = shuffle(x, y)
    return x, y

def create_model():
    ## DENSE NET
    model = Sequential(
        [
            Input(shape=len(x_train[0],)),
            layers.Dense(400, activation='relu'),
            layers.Dense(400, activation='relu'),
            layers.Dense(400, activation='relu'),
            layers.Dense(400, activation='relu'),
            layers.Dense(400, activation='relu'),
            layers.Dense(400, activation='relu'),
            layers.Dense(400, activation='relu'),
            layers.Dense(len(y_train[0]), activation='relu')
        ]
    )
    model.compile(loss='mean_squared_error', optimizer='Adam')
    return model

In [65]:
x, y = load_dataset('data/datasets/Silica_medium.npz')

ValueError: could not broadcast input array from shape (418,) into shape (507,)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [33]:
model = create_model()

In [34]:
# from keras import backend as K
# K.set_value(model.optimizer.learning_rate, 0.0001)
mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='data/models/silica_medium_relu.keras', save_best_only=True,
                                           monitor='val_loss', mode='min', verbose=1, save_weights_only=False,
                                           save_freq='epoch')

reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                                   patience=100, verbose=1, mode='auto')
history = model.fit(np.array(x_train), np.array(y_train),
                    epochs=1000, batch_size=1000, shuffle=True,
                    validation_data=(np.array(x_test), np.array(y_test)), callbacks=[mcp_save, reduce_lr_loss])
plot_loss(history)

Epoch 1/1000
299/300 [============================>.] - ETA: 0s - loss: 0.0654
Epoch 1: val_loss improved from inf to 0.04182, saving model to data/models\silica_medium_relu.keras
300/300 [==============================] - 15s 49ms/step - loss: 0.0653 - val_loss: 0.0418 - lr: 0.0010
Epoch 2/1000
300/300 [==============================] - ETA: 0s - loss: 0.0371
Epoch 2: val_loss improved from 0.04182 to 0.04081, saving model to data/models\silica_medium_relu.keras
300/300 [==============================] - 13s 44ms/step - loss: 0.0371 - val_loss: 0.0408 - lr: 0.0010
Epoch 3/1000
300/300 [==============================] - ETA: 0s - loss: 0.0326
Epoch 3: val_loss improved from 0.04081 to 0.03348, saving model to data/models\silica_medium_relu.keras
300/300 [==============================] - 14s 46ms/step - loss: 0.0326 - val_loss: 0.0335 - lr: 0.0010
Epoch 4/1000
300/300 [==============================] - ETA: 0s - loss: 0.0308
Epoch 4: val_loss did not improve from 0.03348
300/300 [=====

In [77]:
# load model
model = tf.keras.models.load_model('data/models/carbon_medium_relu.keras', custom_objects={'abs': tf.math.abs})

In [18]:
# predict x_train
prediction = model.predict(np.array(x_train))

9375/9375 [==============================] - 26s 3ms/step


In [68]:
# test on x_train
pore_widths = np.load("data/initial kernels/Size_Kernel_Carbon_Adsorption.npy")
pressures = np.load("data/initial kernels/Pressure_Carbon.npy")
NX, NY = 4 , 5
figure, axis = plt.subplots(NX, NY)
for i in range(NX):
    for j in range(NY):
        k = np.random.randint(0, len(x_train))
        x_scale_factor = max(pore_widths)/len(x_train[k])
        axis[i, j].plot(pore_widths/x_scale_factor, prediction[k], marker=".", label=f"Prediction") 
        axis[i, j].plot(pore_widths/x_scale_factor, y_train[k], marker=".", label="Real distribution")
        axis[i, j].plot(x_train[k], label="Isotherm")
        axis[i, j].set_title(f"№ {k}")
        axis[i, j].title.set_size(10)
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plt.legend()
plot()

ValueError: x and y must have same first dimension, but have shapes (130,) and (128,)

In [69]:
# test with test Generator
from tools import TestApp
# gen = Generator(path_s="data/initial kernels/Kernel_Silica_Adsorption.npy",
#                     path_d="data/initial kernels/Kernel_Silica_Desorption.npy",
#                     path_p_d="data/initial kernels/Pressure_Silica.npy",
#                     path_p_s="data/initial kernels/Pressure_Silica.npy",
#                     path_a="data/initial kernels/Size_Kernel_Silica_Adsorption.npy"
#                     )
gen = Generator(path_s="data/initial kernels/Kernel_Carbon_Adsorption.npy",
                              path_d="data/initial kernels/Kernel_Carbon_Desorption.npy",
                              path_p_d="data/initial kernels/Pressure_Carbon.npy",
                              path_p_s="data/initial kernels/Pressure_Carbon.npy",
                              path_a="data/initial kernels/Size_Kernel_Carbon_Adsorption.npy"
                            )
TestApp.App(model, gen)

ValueError: in user code:

    File "C:\Users\ivano\PycharmProjects\isotherm\venv10\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\ivano\PycharmProjects\isotherm\venv10\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ivano\PycharmProjects\isotherm\venv10\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\ivano\PycharmProjects\isotherm\venv10\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\Users\ivano\PycharmProjects\isotherm\venv10\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ivano\PycharmProjects\isotherm\venv10\l

In [82]:
# Test on experimental data
#exp_file = "MCM-41-adsorb"
exp_file_list = ["MCM-41-adsorb", "SBA-15", "SBA-16", "MIL-101", "MIL-101_2"]
exp_data_list = [np.loadtxt(f"data/real/{i}.txt") for i in exp_file_list]
p_exp_list = []
n_s_exp_raw_list = []
for exp_data in exp_data_list:
    p_exp_list.append(exp_data.T[0])
    n_s_exp_raw_list.append(exp_data.T[1])
# p_exp = np.empty(len(exp_data))
# n_exp = np.empty(len(exp_data))
# for i in range(len(exp_data)):  # Загружаем адсорбцию и давления
#     p_exp[i] = exp_data[i][0]
#     n_exp[i] = exp_data[i][1]
# j = np.argmax(n_exp) + 1  # Отделяем адсорбционную ветку
# n_s_exp_raw = n_exp[:j]
# p_exp_sorb = p_exp[:j]

In [83]:
j = 2
plt.plot(p_exp_list[j], n_s_exp_raw_list[j], marker=".", label=exp_file_list[j])
plot()

In [84]:
# интерполируем экспериментальную изотерму под давления кернала
n_s_exp_list = []
for i in range(len(p_exp_list)):
    n_s_exp_list.append(np.interp(gen.pressures_s[40:547], p_exp_list[i], n_s_exp_raw_list[i]))

In [85]:
j = 2
plt.plot(gen.pressures_s[40:547], n_s_exp_list[j], marker=".", label=exp_file_list[j])
plot()

In [86]:
n_s_exp_for_net_list = [pre_process_isotherm(n_s_exp) for n_s_exp in n_s_exp_list]
fit_exp_list = [model.predict(np.array([n_s_exp_for_net])).T for n_s_exp_for_net in n_s_exp_for_net_list]

1/1 [==============================] - 0s 21ms/step


In [87]:
NX, NY = 2 , 3
figure, axis = plt.subplots(NX, NY)
k = 0
for i in range(NX):
    for j in range(NY):
        x_scale_factor = max(gen.a_array)/max(p_exp_list[k])
        y_scale_factor = max(fit_exp_list[k])/max(n_s_exp_raw_list[k])
        axis[i, j].plot(pore_widths, fit_exp_list[k], marker=".", label=f"Distribution") 
        axis[i, j].plot(p_exp_list[k]*x_scale_factor, n_s_exp_raw_list[k]*y_scale_factor, label=f"{exp_file_list[k]}", marker=".")
        axis[i, j].set_title(f"max at {round(gen.a_array[np.argmax(fit_exp_list[k])], 2)} nm")
        axis[i, j].title.set_size(12)
        axis[i, j].legend(loc="upper right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [88]:
### Classic
from inverse import fit_SLSQP
kernel = np.load("data/initial kernels/Kernel_Carbon_Adsorption.npy")
cut_kernel = []
for i in range(len(kernel)):
    cut_kernel.append(kernel[i][40:547])
cut_kernel = np.array(cut_kernel)
fit_classic_list = [fit_SLSQP(adsorption=n_s, kernel=cut_kernel, a_array=pore_widths) for n_s in n_s_exp_list]

In [89]:
NX, NY = 2 , 3
figure, axis = plt.subplots(NX, NY)
k = 0
for i in range(NX):
    for j in range(NY):
        x_scale_factor = max(gen.a_array)/max(p_exp_list[k])
        y_scale_factor = max(fit_classic_list[k].x)/max(n_s_exp_raw_list[k])
        axis[i, j].plot(pore_widths, fit_classic_list[k].x, marker=".", label=f"Distribution") 
        axis[i, j].plot(p_exp_list[k]*x_scale_factor, n_s_exp_raw_list[k]*y_scale_factor, label=f"{exp_file_list[k]}", marker=".")
        axis[i, j].set_title(f"max at {round(gen.a_array[np.argmax(fit_classic_list[k].x)], 2)} nm")
        axis[i, j].title.set_size(12)
        axis[i, j].legend(loc="upper right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [92]:
# compare net and classic
NX, NY = 2 , 3
figure, axis = plt.subplots(NX, NY)
k = 0
for i in range(NX):
    for j in range(NY):
        y_scale_factor = max(fit_classic_list[k].x)/max(fit_exp_list[k])
        axis[i, j].plot(pore_widths, fit_exp_list[k]*y_scale_factor, marker=".", label=f"net") 
        axis[i, j].plot(pore_widths, fit_classic_list[k].x, marker=".", label=f"classic") 
        axis[i, j].set_title(f"{exp_file_list[k]}")
        axis[i, j].title.set_size(12)
        axis[i, j].legend(loc="upper right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
